In [ ]:
import sys
sys.append('../')
from utils.common_utils import read_json


In [ ]:
import os
# 设置HTTP和HTTPS代理
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Qwen2ForCausalLM

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

In [ ]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('qwen/Qwen2.5-Coder-1.5B-Instruct')

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/home/linzhisheng/.cache/modelscope/hub/qwen/Qwen2___5-Coder-1___5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "write a quick sort algorithm."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

/home/linzhisheng/miniconda3/envs/moe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
import math


class multiHeadAttention:
    def __init__(self, hidden_dim, head_num, group_num) -> None:
        
        self.hidden_dim = hidden_dim
        self.head_num = head_num
        self.head_dim = self.hidden_dim // self.head_num
        self.group_num = group_num
        self.key_value_head_num = self.head_num // group_num
        
        self.q = torch.nn.Linear(hidden_dim, self.head_num * self.head_dim)
        self.k = torch.nn.Linear(hidden_dim, self.key_value_head_num * self.head_dim)
        self.v = torch.nn.Linear(hidden_dim, self.key_value_head_num * self.head_dim)
        self.o = torch.nn.Linear(hidden_dim, hidden_dim)

        self.softmax = torch.nn.Softmax(dim=-1)
        self.dropout = torch.nn.Dropout()
    
    def forward(self, x):
        
        bz, seq_len, hidden_dim = x.shape
        
        query = self.q(x).view(bz, seq_len, self.head_num, self.head_dim).transpose(1, 2)
        key = self.k(x).view(bz, seq_len, self.key_value_head_num, self.head_dim).transpose(1, 2)
        value = self.v(x).view(bz, seq_len, self.key_value_head_num, self.head_dim).transpose(1, 2)
        # output = self.o(x).view(bz, seq_len, self.head_num, self.head_dim)
        
        key = torch.repeat_interleave(key, dim=1, repeats=self.group_num)
        value = torch.repeat_interleave(value, dim=1, repeats=self.group_num)
        print(value.shape)
        
        # print(query.shape)
        score = torch.matmul(query, key.transpose(2,3)) / math.sqrt(self.head_dim)
        
        prob = self.softmax(score)
        prob = self.dropout(prob)
        
        output = torch.matmul(prob, value).transpose(1,2).contiguous().view(bz, seq_len, hidden_dim)
        
        output = self.o(output)
        return output

x = torch.randn(12, 24, 768)
attention = multiHeadAttention(768, 2, 2)

output = attention.forward(x)

output.shape





torch.Size([12, 2, 24, 384])


torch.Size([12, 24, 768])

In [ ]:
from transformers